In [1]:
!pip install ase spglib

In [1]:
from ase.io import read, write
from ase.build import make_supercell
import spglib

# CIF 파일 로드
atoms = read("MIL125.cif")

# primitive cell로 변환
lattice = (atoms.cell, atoms.get_scaled_positions(), atoms.get_atomic_numbers())
primitive = spglib.find_primitive(lattice)

if primitive:
    from ase import Atoms
    cell, scaled_positions, numbers = primitive
    atoms_primitive = Atoms(numbers=numbers, cell=cell, scaled_positions=scaled_positions, pbc=True)
    write("MIL125_primitive.cif", atoms_primitive)
    print("[✅] primitive cell 저장 완료!")
else:
    print("❗ primitive cell 변환 실패")


[✅] primitive cell 저장 완료!


In [2]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.cif import CifWriter

structure = Structure.from_file("MIL125.cif")
sga = SpacegroupAnalyzer(structure)

# 가장 단순한 primitive structure
prim_std = sga.get_primitive_standard_structure()
CifWriter(prim_std).write_file("primitive_standard.cif")
print("[✅] primitive_standard.cif 저장 완료!")


[✅] primitive_standard.cif 저장 완료!


In [ ]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

structure = Structure.from_file("MIL-125.cif")
sga = SpacegroupAnalyzer(structure, symprec=1e-3)

# 1) primitive cell
primitive = sga.find_primitive()

# 2) conventional standard cell
conventional = sga.get_conventional_standard_structure()

# 3) primitive + standardized
prim_std = sga.get_primitive_standard_structure()


ValueError: Buffer dtype mismatch, expected 'const int64_t' but got 'long'

In [3]:
from pymatgen.core import Structure, IStructure, Element
import numpy as np
import math
def add_Hs_to_N_in_ring(structure, bond_length=1.01, r_cutoff=5):
    # IStructure → Structure (mutable)
    new_structure = Structure.from_sites(structure.sites)
    coords = np.array([site.coords for site in new_structure])
    species = [site.species_string for site in new_structure]
    new_sites = []

    for i, site in enumerate(new_structure.sites):
        if site.species_string != "N":
            continue

        distances = np.linalg.norm(coords - site.coords, axis=1)
        neighbor_indices = np.where((distances < r_cutoff) & (distances > 0.1))[0]

        C_neighbors_ = [(new_structure[j], coords[j], distances[j]) for j in neighbor_indices if species[j] == "C"]
        C_neighbors = sorted(C_neighbors_, key=lambda x: x[2])[:3]  # 가장 가까운 C 2개

        if len(C_neighbors) == 3:
            vec1 = site.coords - C_neighbors[1][0].coords
            vec2 = site.coords - C_neighbors[2][0].coords

            # 평면 법선 벡터
            normal = np.cross(vec1, vec2)
            normal /= np.linalg.norm(normal)

            # 평균 벡터 방향 (C1, C2 방향 중간)
            mean_vec = (vec1 + vec2) / 2
            mean_vec /= np.linalg.norm(mean_vec)

            # 수소 방향 벡터의 기준 축: 법선벡터(normal)를 중심으로 수소 두 개를 ±30도 회전해서 배치
            angle_deg = 30
            angle_rad = math.radians(angle_deg)

            # 회전축: normal
            def rotate_vector(v, axis, theta):
                """
                Rodrigues' rotation formula
                """
                axis = axis / np.linalg.norm(axis)
                v_rot = (v * math.cos(theta) +
                        np.cross(axis, v) * math.sin(theta) +
                        axis * np.dot(axis, v) * (1 - math.cos(theta)))
                return v_rot

            # 두 수소 벡터 방향
            h_vec1 = rotate_vector(mean_vec, normal, angle_rad)
            h_vec2 = rotate_vector(mean_vec, normal, -angle_rad)

            # 수소 좌표 계산
            h1_pos = site.coords + bond_length * h_vec1
            h2_pos = site.coords + bond_length * h_vec2

            new_sites.append(("H", h1_pos))
            new_sites.append(("H", h2_pos))

            print(new_sites)
    for sp, pos in new_sites:
        frac_coords = new_structure.lattice.get_fractional_coords(pos)
        new_structure.append(sp, frac_coords, coords_are_cartesian=False)

    return new_structure, coords, distances

input_cif = "MIL125_primitive.cif"
structure = IStructure.from_file(input_cif)

new_structure, coords, distances = add_Hs_to_N_in_ring(structure)

output_cif = "MIL125_NH2_added.cif"
new_structure.to(fmt="cif", filename=output_cif)
print(f"✅ 완료: {output_cif} 저장됨.")



[('H', array([1.83536207, 8.72932423, 1.18482885])), ('H', array([1.19057446, 8.6223089 , 1.95482669]))]
[('H', array([1.83536207, 8.72932423, 1.18482885])), ('H', array([1.19057446, 8.6223089 , 1.95482669])), ('H', array([ 0.37352005, 14.09384324, -5.06080127])), ('H', array([ 0.54531643, 13.46986495, -4.28540647]))]
[('H', array([1.83536207, 8.72932423, 1.18482885])), ('H', array([1.19057446, 8.6223089 , 1.95482669])), ('H', array([ 0.37352005, 14.09384324, -5.06080127])), ('H', array([ 0.54531643, 13.46986495, -4.28540647])), ('H', array([9.87725852, 6.00166476, 7.7120888 ])), ('H', array([10.72398222,  5.496446  ,  7.49319847]))]
[('H', array([1.83536207, 8.72932423, 1.18482885])), ('H', array([1.19057446, 8.6223089 , 1.95482669])), ('H', array([ 0.37352005, 14.09384324, -5.06080127])), ('H', array([ 0.54531643, 13.46986495, -4.28540647])), ('H', array([9.87725852, 6.00166476, 7.7120888 ])), ('H', array([10.72398222,  5.496446  ,  7.49319847])), ('H', array([ 9.52235925, 11.4014539

In [5]:
from pymatgen.core import Structure, IStructure, Element
import numpy as np
import math

def rotate_vector(v, axis, theta):
    """
    Rodrigues' rotation formula to rotate vector `v` around `axis` by angle `theta` (radians)
    """
    axis = axis / np.linalg.norm(axis)
    v_rot = (v * math.cos(theta) +
             np.cross(axis, v) * math.sin(theta) +
             axis * np.dot(axis, v) * (1 - math.cos(theta)))
    return v_rot

def add_Hs_sp3_from_CN(structure, bond_length=1.01, r_cutoff=5):
    # IStructure → Structure (mutable)
    new_structure = Structure.from_sites(structure.sites)
    coords = np.array([site.coords for site in new_structure])
    species = [site.species_string for site in new_structure]
    new_sites = []

    for i, site in enumerate(new_structure.sites):
        if site.species_string != "N":
            continue

        # 주변 원자 탐색
        distances = np.linalg.norm(coords - site.coords, axis=1)
        neighbor_indices = np.where((distances < r_cutoff) & (distances > 0.1))[0]

        # 가까운 C 원자 최대 2개
        C_neighbors_ = [(new_structure[j], coords[j], distances[j]) for j in neighbor_indices if species[j] == "C"]
        C_neighbors = sorted(C_neighbors_, key=lambda x: x[2])[:2]

        if len(C_neighbors) < 2:
            continue

        # C–N 방향 벡터 (가장 가까운 C 기준)
        v_cn = site.coords - C_neighbors[0][0].coords
        v_cn /= np.linalg.norm(v_cn)

        # 평면 법선 벡터 (N-C1, N-C2 이용)
        vec1 = site.coords - C_neighbors[0][0].coords
        vec2 = site.coords - C_neighbors[1][0].coords
        normal = np.cross(vec1, vec2)
        normal /= np.linalg.norm(normal)

        # sp³ 각도 (109.5도 기준) → 대칭 회전
        angle_rad = math.radians(109.5 / 2)  # 반각으로 회전
        h_vec1 = rotate_vector(-v_cn, normal, angle_rad)
        h_vec2 = rotate_vector(-v_cn, normal, -angle_rad)

        # 수소 위치
        h1_pos = site.coords + bond_length * h_vec1
        h2_pos = site.coords + bond_length * h_vec2

        new_sites.append(("H", h1_pos))
        new_sites.append(("H", h2_pos))
        print(h1_pos, h2_pos)
    # 구조에 수소 추가 (분수좌표 변환)
    for sp, pos in new_sites:
        frac_coords = new_structure.lattice.get_fractional_coords(pos)
        new_structure.append(sp, frac_coords, coords_are_cartesian=False)

    return new_structure

# ⬇️ 실행
input_cif = "MIL125_primitive.cif"
structure = IStructure.from_file(input_cif)

# H 추가
new_structure = add_Hs_sp3_from_CN(structure)

# 저장
output_cif = "MIL125_NH2_sp3_added.cif"
new_structure.to(fmt="cif", filename=output_cif)

print(f"✅ 완료: {output_cif} 저장됨.")


[2.97191557 7.84534802 1.60188977] [1.90944432 7.68281386 2.85327709]
[ 0.03355247 12.29310727 -5.7472793 ] [ 0.18127264 13.9360635  -5.73696848]
[9.87279683 5.17740971 8.5020369 ] [10.53579377  5.57942305  7.04599669]
[ 7.591946   11.09986583 -1.81882771] [ 9.0189061  11.91607393 -1.95602089]
[9.5193084  7.41692987 0.29081841] [10.30605054  7.99941538  1.61859106]
[8.0106668  1.14570654 7.29755548] [9.29811721 0.170241   6.96263586]
✅ 완료: MIL125_NH2_sp3_added.cif 저장됨.


In [7]:
from pymatgen.core import Structure, IStructure, Element
import numpy as np
import math

def add_Hs_simple_sp3(structure, bond_length=1.01, r_cutoff=5, spread_angle_deg=30):
    new_structure = Structure.from_sites(structure.sites)
    coords = np.array([site.coords for site in new_structure])
    species = [site.species_string for site in new_structure]
    new_sites = []

    for i, site in enumerate(new_structure.sites):
        if site.species_string != "N":
            continue

        # 거리 기반으로 주변 C 탐색
        distances = np.linalg.norm(coords - site.coords, axis=1)
        neighbor_indices = np.where((distances < r_cutoff) & (distances > 0.1))[0]
        C_neighbors = sorted(
            [(new_structure[j], distances[j]) for j in neighbor_indices if species[j] == "C"],
            key=lambda x: x[1]
        )

        if len(C_neighbors) == 0:
            continue

        # C → N 벡터
        v_cn = site.coords - C_neighbors[0][0].coords
        v_cn /= np.linalg.norm(v_cn)

        # v_cn에 수직인 벡터 만들기
        if abs(v_cn[0]) < 0.9:
            temp = np.array([1, 0, 0])
        else:
            temp = np.array([0, 1, 0])
        v_perp = np.cross(v_cn, temp)
        v_perp /= np.linalg.norm(v_perp)

        # 수소 방향 벡터 (벌어지는 각도 설정)
        theta = math.radians(spread_angle_deg)
        h_vec1 = -v_cn + math.tan(theta) * v_perp
        h_vec2 = -v_cn - math.tan(theta) * v_perp
        h_vec1 /= np.linalg.norm(h_vec1)
        h_vec2 /= np.linalg.norm(h_vec2)

        h1_pos = site.coords + bond_length * (-1) * h_vec1
        h2_pos = site.coords + bond_length * (-1) *h_vec2

        new_sites.append(("H", h1_pos))
        new_sites.append(("H", h2_pos))
        print("💧 Hs for N at", site.coords, "\n→", h1_pos, "\n→", h2_pos)

    # 구조에 수소 추가
    for sp, pos in new_sites:
        frac_coords = new_structure.lattice.get_fractional_coords(pos)
        new_structure.append(sp, frac_coords, coords_are_cartesian=False)

    return new_structure

# 실행
input_cif = "MIL125_primitive.cif"
structure = IStructure.from_file(input_cif)

new_structure = add_Hs_simple_sp3(structure)

output_cif = "MIL125_NH2_simple_added.cif"
new_structure.to(fmt="cif", filename=output_cif)
print(f"✅ 완료: {output_cif} 저장됨.")


💧 Hs for N at [2.07174865 8.12876598 1.96171441] 
→ [1.51815507 8.9734854  1.97083809] 
→ [1.51815507 8.37849042 1.15470025]
💧 Hs for N at [ 0.26175892 13.09718193 -5.18028216] 
→ [ 0.49336072 12.56630956 -4.35285534] 
→ [ 0.49336072 13.57582537 -4.32158485]
💧 Hs for N at [9.83689613 5.01373128 7.50603464] 
→ [9.28560155 4.76554426 6.696975  ] 
→ [9.28560155 4.16747424 7.51086212]
💧 Hs for N at [ 8.59653966 11.00372668 -1.85943372] 
→ [ 9.03336524 10.21910362 -2.32165667] 
→ [ 9.03336524 10.27508262 -1.31320917]
💧 Hs for N at [9.45773945 8.05286806 1.0730536 ] 
→ [8.77508695 8.40610338 1.7282714 ] 
→ [8.77508695 8.73408647 0.77300856]
💧 Hs for N at [8.93239922 0.86380377 7.59928412] 
→ [9.34955821 0.71020198 8.50619364] 
→ [9.34955821 1.63511501 8.10044015]
✅ 완료: MIL125_NH2_simple_added.cif 저장됨.
